In [5]:
import pandas as pd
import numpy as np
from scipy.stats import (
    f_oneway,
    ks_2samp,
    lognorm,
    beta as beta_dist,
    gamma as gamma_dist,
    poisson,
    geom,
    chisquare,
)
from statsmodels.stats.multitest import multipletests
from matplotlib import pyplot as plt

# Load the real data
df_real = pd.read_csv('Raw Data_GeneSpring.txt', sep='\t')
df_real = df_real.iloc[:, :-3]  # Drop last 3 unnecessary columns

# Extract gene expression data
gene_expression = df_real.iloc[:, 1:]  # Assuming first column is 'ProbeName'

# Compute overall statistics
overall_mean = gene_expression.values.mean()
overall_std = gene_expression.values.std()
overall_min = gene_expression.values.min()
overall_max = gene_expression.values.max()

print(f"Overall Mean: {overall_mean:.4f}")
print(f"Overall Std Dev: {overall_std:.4f}")
print(f"Overall Min: {overall_min:.4f}")
print(f"Overall Max: {overall_max:.4f}\n")

# Fit log-normal distribution
log_data = np.log(gene_expression.values + 1e-6)  # Add small value to avoid log(0)
mu_lognorm = log_data.mean()
sigma_lognorm = log_data.std()
print(f"Log-Normal Distribution Parameters:")
print(f" - mu: {mu_lognorm:.4f}")
print(f" - sigma: {sigma_lognorm:.4f}\n")

# Fit beta distribution
epsilon = 1e-6
if overall_max != overall_min:
    normalized_data = (gene_expression.values - overall_min) / (overall_max - overall_min)
    normalized_data = np.clip(normalized_data, epsilon, 1 - epsilon)
    # Flatten the data for fitting
    normalized_data_flat = normalized_data.flatten()
    a_beta, b_beta, loc_beta, scale_beta = beta_dist.fit(normalized_data_flat, floc=0, fscale=1)
    print(f"Beta Distribution Parameters:")
    print(f" - alpha (a): {a_beta:.4f}")
    print(f" - beta (b): {b_beta:.4f}\n")
else:
    raise ValueError("All gene expression values are identical; cannot perform normalization.")

# Fit gamma distribution with shifted data
epsilon_gamma = 1e-6  # Define a small constant to shift data
shifted_data = gene_expression.values + epsilon_gamma

# Check if all shifted_data > 0
if np.any(shifted_data <= 0):
    raise ValueError("Shifted data contains non-positive values; cannot fit Gamma distribution.")

# Fit Gamma distribution
shape_gamma, loc_gamma, scale_gamma = gamma_dist.fit(shifted_data, floc=0)
print(f"Gamma Distribution Parameters (Shifted Data):")
print(f" - shape (k): {shape_gamma:.4f}")
print(f" - scale (θ): {scale_gamma:.4f}\n")

# Continue with other distributions as in your original code...

# Example: Poisson and Geometric distributions
# Note: These are discrete distributions; fitting them to continuous gene expression data may not be appropriate
lambda_poisson = overall_mean
print(f"Poisson Distribution Parameter:")
print(f" - lambda: {lambda_poisson:.4f}\n")

p_geometric = 1 / (overall_mean + 1)
print(f"Geometric Distribution Parameter:")
print(f" - p: {p_geometric:.4f}\n")

# Exponential distribution
scale_exponential = overall_mean
print(f"Exponential Distribution Parameter:")
print(f" - scale: {scale_exponential:.4f}\n")

# Chi-Squared distribution
df_chi_squared = overall_mean
print(f"Chi-Squared Distribution Parameter:")
print(f" - degrees of freedom (df): {df_chi_squared:.4f}\n")

# Normal distribution
mean_normal = overall_mean
std_normal = overall_std
print(f"Normal Distribution Parameters:")
print(f" - mean: {mean_normal:.4f}")
print(f" - standard deviation: {std_normal:.4f}\n")

# Uniform distribution
min_uniform = overall_min
max_uniform = overall_max
print(f"Uniform Distribution Parameters:")
print(f" - low: {min_uniform:.4f}")
print(f" - high: {max_uniform:.4f}\n")


Overall Mean: 6.8050
Overall Std Dev: 4.1252
Overall Min: 0.0000
Overall Max: 19.5638

Log-Normal Distribution Parameters:
 - mu: 0.0549
 - sigma: 5.0844

Beta Distribution Parameters:
 - alpha (a): 0.3942
 - beta (b): 1.0338

Gamma Distribution Parameters (Shifted Data):
 - shape (k): 0.3594
 - scale (θ): 18.9329

Poisson Distribution Parameter:
 - lambda: 6.8050

Geometric Distribution Parameter:
 - p: 0.1281

Exponential Distribution Parameter:
 - scale: 6.8050

Chi-Squared Distribution Parameter:
 - degrees of freedom (df): 6.8050

Normal Distribution Parameters:
 - mean: 6.8050
 - standard deviation: 4.1252

Uniform Distribution Parameters:
 - low: 0.0000
 - high: 19.5638

